In [11]:
%matplotlib inline

from qiskit import *
from qiskit.tools.monitor import * 
from qiskit.providers.ibmq import least_busy
import numpy as np


In [12]:
# the CHSH game

def CHSH_circuit(x,y,a0=0, a1=np.pi/2, b0=np.pi/4, b1= -np.pi/4):
    
    # x is the bit received by Alice
    # y is the bit received by Bob
    # a0 is the angles used by Alice if she gets 0
    # a1 is the angle used by Alice is she gets 1
    # b0 is the angle used by Bob if he gets 0
    # b1 is the angle used by Bob if he gets 1
    
    
    
    
    circ= QuantumCircuit(2,2)
    
    # make a bell pair
    circ.h(0)
    circ.cx(0,1)
    
    if(x==0):
        circ.ry(a0,0)
    else:
        circ.ry(a1,0)
        
    if(y==0):
        circ.ry(b0,1)
    else:
        circ.ry(b1,1)
        
        
        # measurement
    circ.measure(range(2), range(2))
    
    return circ

In [17]:
def winning_probability(backend,shots = 8192, a0=0, a1=np.pi/2, b0=np.pi/4, b1= -np.pi/4):
    total = 0
    circuits = [CHSH_circuit(0,0,a0,a1,b0,b1), CHSH_circuit(0,1,a0,a1,b0,b1),CHSH_circuit(1,0,a0,a1,b0,b1),CHSH_circuit(1,1,a0,a1,b0,b1)]
    job = execute(circuits,backend=backend, shots = shots)
    
    # for the first three circuits, the winning condition is when both the outputs are equal
    
    for qc in circuits[0:3]:
        counts = job.result().get_counts(qc)
        if ('00' in counts):
            total += counts['00']
        if ('11' in counts):
            total += counts['11']
        # for the fourth 
        counts = job.result().get_counts(circuits[3])
        if('01' in counts):
            total += counts['01']
        if('10' in counts):
            total += counts['10']
            
        return total/(2*shots)

In [18]:
backend = Aer.get_backend('qasm_simulator')

print(winning_probability(backend))

0.85137939453125


In [19]:
# run on a quantum computer

provider = IBMQ.load_account()

backend = least_busy(provider.backends(operational = True, simulator = False, status_msg='active',
                                      filters=lambda x: x.configuration().n_qubits >1))
print("we are executing on ...", backend)

print(winning_probability(backend))

we are executing on ... ibmq_lima
0.8218994140625
